In [ ]:
path_consultoria = '/content/drive/MyDrive/Consultoria/Final'

In [ ]:
!cp -r {path_consultoria}/UR5Inv /content/

In [ ]:
!pip install optuna

In [ ]:
from UR5Inv.RF import RFInversaUR5

In [ ]:
def comparar_modelos_rf(nombre_modelo, ruta_datos, modelo, ruta_csv=None):
    """
    Crea o actualiza un archivo CSV con métricas de comparación para modelos RF.
    Aprovecha las métricas ya calculadas por el modelo sin recalcularlas.

    Parámetros:
        nombre_modelo (str): Nombre identificador del modelo (ej: "Aleatorio_RF").
        ruta_datos (str): Ruta al archivo de datos utilizado para entrenar el modelo.
        modelo (RFInversaUR5 o RFInversaUR5Simple): Instancia del modelo RF entrenado.
        ruta_csv (str, opcional): Ruta donde guardar el archivo CSV.
    """
    import pandas as pd
    import numpy as np
    import os
    import time

    # Determinar ruta del CSV si no se especifica
    if ruta_csv is None:
        directorio_datos = os.path.dirname(ruta_datos)
        ruta_csv = os.path.join(directorio_datos, "comparacion_modelos.csv")

    # Obtener resultados de la evaluación del modelo (sin recalcular)
    # Asumimos que el modelo ya ha sido evaluado

    # Calcular tiempo de inferencia promedio (100 muestras)
    tiempos = []
    n_muestras = min(100, len(modelo.X))
    indices = np.random.choice(len(modelo.X), n_muestras, replace=False)

    for idx in indices:
        x = modelo.X[idx:idx+1]
        inicio = time.time()
        modelo.predecir(x)
        fin = time.time()
        tiempos.append(fin - inicio)

    tiempo_inf = np.mean(tiempos) * 1000  # ms

    # Calcular tamaño aproximado del modelo
    if hasattr(modelo, 'n_estimators'):
        n_trees = modelo.n_estimators
    else:
        n_trees = modelo.study.best_params.get('n_estimators', 100)

    parametros = n_trees * modelo.output_dim * 100  # Aproximación

    # Crear diccionario con las métricas de los resultados existentes
    metricas = {
        'modelo': nombre_modelo,
        'MSE_train': modelo.train_loss[0] if hasattr(modelo, 'train_loss') else None,
        'MSE_test': modelo.test_loss[0] if hasattr(modelo, 'test_loss') else None,
        'MSE_global': np.mean(modelo.train_loss) if hasattr(modelo, 'train_loss') else None,
        'tamaño_modelo': parametros,
        'tiempo_inf': tiempo_inf
    }

    # Añadir errores L1 por articulación (si están disponibles)
    if hasattr(modelo, 'train_loss_l1'):
        for i, error in enumerate(modelo.train_loss_l1):
            metricas[f'Errorq{i}L1'] = error

    # Crear o actualizar el archivo CSV
    try:
        df_existente = pd.read_csv(ruta_csv)
        if nombre_modelo in df_existente['modelo'].values:
            df_existente = df_existente[df_existente['modelo'] != nombre_modelo]
        df_nueva_fila = pd.DataFrame([metricas])
        df_final = pd.concat([df_existente, df_nueva_fila], ignore_index=True)
    except (FileNotFoundError, pd.errors.EmptyDataError):
        df_final = pd.DataFrame([metricas])

    df_final.to_csv(ruta_csv, index=False)
    print(f"Comparación actualizada en: {ruta_csv}")

    return df_final

In [ ]:
import numpy as np
import pandas as pd
from cuml.ensemble import RandomForestRegressor as cuRF
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import time
import os
import joblib
import optuna
import optuna.visualization.matplotlib as vis


## Datos comandos

In [ ]:

datos = r"/content/drive/MyDrive/Consultoria/Final/Data/2dos/datos_comandos.csv"

modelo = RFInversaUR5(
    ruta_datos=datos,
    test_size=0.25,
    random_state=42,
)
print(f"Datos cargados: {modelo.X.shape[0]} muestras")
print(f"Dimensiones: entrada={modelo.input_dim}, salida={modelo.output_dim}")

# Ejecutamos la optimización con Optuna
print("\nOptimizando hiperparámetros con Optuna...")
modelo.optimizar(n_trials=30)

# Entrenamos el mejor modelo encontrado
print("\nEntrenando el mejor modelo...")
modelo.entrenar_mejor_modelo()

# Visualizaciones
print("\nGenerando gráficos...")
# Gráfico de pérdidas
modelo.graficar_perdidas()

# Evaluación y resumen
print("\nEvaluación del modelo:")
modelo.summary()

# Guardar modelo y resultados
nombre_carpeta = "Comandos_RF"
modelo.guardar(nombre_carpeta)


In [ ]:
# Guardar métricas comparativas
comparar_modelos_rf(
    nombre_modelo="Comandos_RF",
    ruta_datos=datos,
    modelo=modelo
)

In [ ]:


import locale
locale.getpreferredencoding = lambda: 'UTF-8'

!export LANG=C.UTF-8
!export LC_ALL=C.UTF-8
!export LC_CTYPE=C.UTF-8

!cp -r {nombre_carpeta} /content/drive/MyDrive/Consultoria/Final

## Datos Aleatorios

In [ ]:

datos = r"/content/drive/MyDrive/Consultoria/Final/Data/3ros/Aleatorio.csv"

modelo = RFInversaUR5(
    ruta_datos=datos,
    test_size=0.15,
    random_state=42,
)
print(f"Datos cargados: {modelo.X.shape[0]} muestras")
print(f"Dimensiones: entrada={modelo.input_dim}, salida={modelo.output_dim}")

# Ejecutamos la optimización con Optuna
print("\nOptimizando hiperparámetros con Optuna...")
modelo.optimizar(n_trials=30)

# Entrenamos el mejor modelo encontrado
print("\nEntrenando el mejor modelo...")
modelo.entrenar_mejor_modelo()

# Visualizaciones
print("\nGenerando gráficos...")
# Gráfico de pérdidas
modelo.graficar_perdidas()

# Evaluación y resumen
print("\nEvaluación del modelo:")
modelo.summary()

# Guardar modelo y resultados
nombre_carpeta = "Aleatorios_RF"
modelo.guardar(nombre_carpeta)



In [ ]:
# Guardar métricas comparativas
comparar_modelos_rf(
    nombre_modelo="Aleatorio_RF",
    ruta_datos=datos,
    modelo=modelo
)


!cp -r {nombre_carpeta} /content/drive/MyDrive/Consultoria/Final

## Trayectorias

In [ ]:

datos = r"/content/drive/MyDrive/Consultoria/Final/Data/Gen1/openaxes-example-robot-dataset/robot/Complete.csv"

modelo = RFInversaUR5(
    ruta_datos=datos,
    test_size=0.25,
    random_state=42,
)
print(f"Datos cargados: {modelo.X.shape[0]} muestras")
print(f"Dimensiones: entrada={modelo.input_dim}, salida={modelo.output_dim}")

# Ejecutamos la optimización con Optuna
print("\nOptimizando hiperparámetros con Optuna...")
modelo.optimizar(n_trials=50)

# Entrenamos el mejor modelo encontrado
print("\nEntrenando el mejor modelo...")
modelo.entrenar_mejor_modelo()

# Visualizaciones
print("\nGenerando gráficos...")
# Gráfico de pérdidas
modelo.graficar_perdidas()

# Evaluación y resumen
print("\nEvaluación del modelo:")
modelo.summary()

# Guardar modelo y resultados
nombre_carpeta = "Trayectorias_RF"
modelo.guardar(nombre_carpeta)

# Guardar métricas comparativas
comparar_modelos_rf(
    nombre_modelo="Trayectorias_RF",
    ruta_datos=datos,
    modelo=modelo
)

!cp -r {nombre_carpeta} /content/drive/MyDrive/Consultoria/Final

In [ ]:
!cp -r {nombre_carpeta} /content/drive/MyDrive/Consultoria/Final

## PesosVel

In [ ]:

datos = r"/content/drive/MyDrive/Consultoria/Final/Data/Gen2/NISTdata.csv"

modelo = RFInversaUR5(
    ruta_datos=datos,
    test_size=0.30,
    random_state=42,
)
print(f"Datos cargados: {modelo.X.shape[0]} muestras")
print(f"Dimensiones: entrada={modelo.input_dim}, salida={modelo.output_dim}")

# Ejecutamos la optimización con Optuna
print("\nOptimizando hiperparámetros con Optuna...")
modelo.optimizar(n_trials=18)

# Entrenamos el mejor modelo encontrado
print("\nEntrenando el mejor modelo...")
modelo.entrenar_mejor_modelo()

# Visualizaciones
print("\nGenerando gráficos...")
# Gráfico de pérdidas
modelo.graficar_perdidas()

# Evaluación y resumen
print("\nEvaluación del modelo:")
modelo.summary()

# Guardar modelo y resultados
nombre_carpeta = "PesosVel_RF"
modelo.guardar(nombre_carpeta)

# Guardar métricas comparativas
comparar_modelos_rf(
    nombre_modelo="PesosVel_RF",
    ruta_datos=datos,
    modelo=modelo
)

import locale
locale.getpreferredencoding = lambda: 'UTF-8'

!export LANG=C.UTF-8
!export LC_ALL=C.UTF-8
!export LC_CTYPE=C.UTF-8


!cp -r {nombre_carpeta} /content/drive/MyDrive/Consultoria/Final

In [ ]:
!cp -r {nombre_carpeta} /content/drive/MyDrive/Consultoria/Final